In [1]:
# ================================
# 국민 동의 청원 AI 모델 - 예측 코드
# ================================

# 1️⃣ 라이브러리 설치 및 환경 설정
import os
print("📂 현재 작업 디렉토리:", os.getcwd())

# 필요한 라이브러리 설치
!pip install --upgrade transformers torch torchvision torchaudio
!pip install pandas numpy transformers torch tqdm

# 2️⃣ 저장된 모델 불러오기
import pickle

try:
    with open("청원_예측모델.pkl", "rb") as model_file:
        model = pickle.load(model_file)
    print("✅ 모델 불러오기 성공!")
except Exception as e:
    print(f"❌ 모델 불러오기 실패: {e}")
    exit()

# 3️⃣ 진행 중 청원 데이터 로드
import pandas as pd

file_path = "진행중국민동의청원현황1.csv"

try:
    진행_청원 = pd.read_csv(file_path, encoding="utf-8")
    print("✅ 진행 중 청원 데이터 로드 성공!")
except Exception as e:
    print(f"❌ 진행 중 청원 데이터 로드 실패: {e}")
    진행_청원 = pd.DataFrame(columns=["청원제목", "청원내용"])  # 빈 DataFrame 생성
    print("빈 DataFrame을 생성했습니다.")

# 4️⃣ 제출 주체 정보 추가 (국민 vs. 국회의원)
진행_청원["제출주체"] = 진행_청원["청원제목"].apply(lambda x: "국회의원" if "법안" in str(x) else "국민")
진행_청원["제출주체"] = 진행_청원["제출주체"].map({"국민": 0, "국회의원": 1})

# 5️⃣ KoBERT 임베딩 변환 (청원 제목 + 본문)
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model_bert = AutoModel.from_pretrained("monologg/kobert")

def get_bert_embedding(texts):
    embeddings = []
    for text in tqdm(texts, desc="BERT 임베딩 생성"):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        with torch.no_grad():
            outputs = model_bert(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)
    return np.array(embeddings)

# 6️⃣ 크롤링된 ‘청원 요지’ 데이터 로드
try:
    크롤링된_청원 = pd.read_csv("청원_처리_현황_크롤링완료.csv", encoding="cp949")
    진행_청원 = 진행_청원.merge(크롤링된_청원[["청원제목", "청원내용"]], on="청원제목", how="left")
    print("✅ 크롤링된 청원 요지 데이터 추가 완료!")
except Exception as e:
    print(f"❌ 크롤링된 청원 요지 데이터 로드 실패: {e}")

# 7️⃣ BERT 임베딩 변환
test_texts = (진행_청원["청원제목"] + " " + 진행_청원["청원내용"].fillna("")).astype(str).tolist()
test_embeddings = get_bert_embedding(test_texts)

# 8️⃣ 승인 확률 예측
try:
    X_test = np.hstack((test_embeddings, 진행_청원[["제출주체"]].values))
    진행_청원["승인 확률"] = model.predict_proba(X_test)[:, 1] * 100  # 확률을 퍼센트로 변환

    print("✅ 승인 확률 예측 성공!")

    # 예측 결과 저장
    진행_청원[["청원제목", "승인 확률"]].to_csv("예측결과.csv", index=False, encoding="utf-8-sig")
    print("✅ 예측 결과 저장 완료! (예측결과.csv)")
except Exception as e:
    print(f"❌ 승인 확률 예측 실패: {e}")
    exit()

📂 현재 작업 디렉토리: /Users/sky/VSCode/petition/AI

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✅ 모델 불러오기 성공!
✅ 진행 중 청원 데이터 로드 성공!


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


❌ 크롤링된 청원 요지 데이터 로드 실패: [Errno 2] No such file or directory: '청원_처리_현황_크롤링완료.csv'


BERT 임베딩 생성: 100%|████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 15.51it/s]

❌ 승인 확률 예측 실패: 'XGBRegressor' object has no attribute 'predict_proba'
